In [56]:
import pypdfium2 as pdfium
import re
import pandas as pd
from ipynb.fs.defs.BERT_Embeddings import get_text_embedding
import pickle
import openai
import time
import os
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

pdf = pdfium.PdfDocument("../02450_Book.pdf")

In [37]:
# Segments of text-sections
text_seg = ""
for page in pdf:
    textpage = page.get_textpage()
    text_seg += "\n".join(textpage.get_text_range().splitlines())

# Make sections
sections = re.split('^\d+\.\d+\.?\d* [a-zA-Z ]+.$', text_seg, flags=re.MULTILINE)

In [ ]:
#Get embeddings
embeddings_seg = [get_text_embedding(section) for section in sections]

In [2]:
with open('./df_pickle/sections.pkl', 'wb') as fp:
    pickle.dump(embeddings_seg, fp)

NameError: name 'embeddings_seg' is not defined

In [8]:
# Segments of sentences
text_sen = ""
for page in pdf:
    textpage = page.get_textpage()
    text_sen += " ".join(textpage.get_text_range().splitlines())

# Make sure no empty sentences are included
sentences = list(filter(None, text_sen.split('.')))

In [ ]:
#Get embeddings
embeddings_sen = [get_text_embedding(sentence) for sentence in sentences]

In [3]:
with open('./df_pickle/sentences.pkl', 'wb') as fp:
    pickle.dump(embeddings_sen, fp)

NameError: name 'embeddings_sen' is not defined

In [72]:
with open('./df_pickle/df_context.pkl', 'rb') as f:
    sec_data = pickle.load(f)

combinations = pd.DataFrame(columns=['context', 'question'])

In [73]:
# Ask ChatGPT to make questions to sections
req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()
# For each pair of questions
for i in range(0, len(sec_data['context'])):
    context = sec_data['context'][i]
    
    # Check rate limit
    req_per_min += 1
    while req_per_min>=max_req_per_min:
        time_stamp = time.time()
        if int(time.time()-last_time_stamp) > 60:
            last_time_stamp = time_stamp
            req_per_min = 0
        else:
            time.sleep(10)
        
    # Ask the oracle for label for the context and two questions
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[
            {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
            {"role": "user", "content": f'Please Generate 20 questions CONTEXT: ```{context}```'}
            ]
        )
    except Exception as e:
        print("OPENAI_ERROR:",str(e))
        continue

    # Parse the response to get labels
    res = completion.choices[0].message.content

    # Split and remove index
    split = res.splitlines()
    questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
    #print(questions)
    
    # If 20 questions are not found, assume something went wrong and skip this iteration
    if len(questions) != 20:
        print(f'[Error] questions not generated for {i}')
        continue
    
    # Save data into a dataframe
    combinations = combinations.append([{'context': context, 'question': question} for question in questions])
    print(f"Questions from round {i} saved!")

C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


Questions from round 0 saved!


C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


Questions from round 1 saved!


C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


...

C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


Questions from round 157 saved!


C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


Questions from round 158 saved!
Questions from round 159 saved!


C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\3700690808.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


In [89]:
# Ask ChatGPT to make questions to sections
req_per_min = 0
max_req_per_min = 3
last_time_stamp = time.time()

# For the single wrong return
context = sec_data['context'][122]
    
# Check rate limit
req_per_min += 1
while req_per_min>=max_req_per_min:
    time_stamp = time.time()
    if int(time.time()-last_time_stamp) > 60:
        last_time_stamp = time_stamp
        req_per_min = 0
    else:
        time.sleep(10)
    
# Ask the oracle for label for the context and two questions
try:
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=[
        {"role":"system", "content":"You are a system designed to generate questions that can be answered using ANY part of a provided text, and which are specific to it."},
        {"role": "user", "content": f'Please Generate a question CONTEXT: ```{context}```'}
        ]
    )
except Exception as e:
    print("OPENAI_ERROR:",str(e))

# Parse the response to get labels
res = completion.choices[0].message.content

# Split and remove index
split = res.splitlines()
questions = [re.sub(r'^\d+\.\s', '', q) for q in split]
#print(questions)

# If 20 questions are not found, assume something went wrong and skip this iteration
if len(questions) != 1:
    print(f'[Error] questions not generated')
else:
    # Save data into a dataframe
    combinations = combinations.append([{'context': context, 'question': question} for question in questions])
    print(f"Questions saved!")

Questions saved!


C:\Users\mathi\AppData\Local\Temp\ipykernel_3092\1766648177.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combinations = combinations.append([{'context': context, 'question': question} for question in questions])


In [114]:
combinations

,context,question
0,How can we build intelligent machines? More t...,What was the subject of Alan Turing's famous e...
1,How can we build intelligent machines? More t...,How did Alan Turing suggest we approach the qu...
2,How can we build intelligent machines? More t...,What did Alan Turing propose as an alternative...
3,How can we build intelligent machines? More t...,What did Turing suggest we should do instead o...
4,How can we build intelligent machines? More t...,How did Turing propose we should build a human...
...,...,...
15,Our third measure of cluster similarity is ba...,What is the Gaussian mixture-model used for?
16,Our third measure of cluster similarity is ba...,What is the EM algorithm used for in the conte...
17,Our third measure of cluster similarity is ba...,Who is credited with first using mixture model...
18,Our third measure of cluster similarity is ba...,When was the EM algorithm first named?


In [98]:
with open('./df_pickle/question-context-20.pkl', 'wb') as fp:
    pickle.dump(combinations, fp)

In [115]:
n_over = 0
for count in combinations['question'].value_counts():
    if count > 1:
        n_over += 1
print(n_over)

0


In [99]:
combinations.drop_duplicates(subset=['question'], inplace=True)

In [116]:
combinations['context'].value_counts().min()

17

In [117]:
n_under = 0
for count in combinations['context'].value_counts():
    if count < 20:
        n_under += 1
print(n_under)

29


In [118]:
with open('./df_pickle/question-context-!dublicates.pkl', 'wb') as fp:
    pickle.dump(combinations, fp)